In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
import gmaps.datasets
# Import the API key.
from config import g_key

# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [2]:
# Store the Challenge CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/WeatherPy_challenge.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,description,Rain Inches (last 3 hrs),Snow Inches (last 3 hrs)
0,0,Port Alfred,ZA,2020-04-19 15:54:32,-33.59,26.89,81.54,52,39,17.45,scattered clouds,0.0,0.0
1,1,Grindavik,IS,2020-04-19 15:49:22,63.84,-22.43,46.99,100,90,23.04,light intensity drizzle,0.0,0.0
2,2,Shimoda,JP,2020-04-19 15:54:43,34.67,138.95,55.00,84,100,1.99,overcast clouds,0.0,0.0
3,3,Ushuaia,AR,2020-04-19 15:51:46,-54.80,-68.30,44.60,75,75,1.10,broken clouds,0.0,0.0
4,4,Kapaa,US,2020-04-19 15:53:45,22.08,-159.32,69.80,83,90,5.82,overcast clouds,0.0,0.0


In [4]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
rain_check=input("Do you want it to be raining? (yes/no) ")
snow_check=input("Do you want it to be snowing? (yes/no) ")

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 70
Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


In [5]:
#Filter the DataFrame for minimum and maximum temperature preferences and prompt for rain/snow
preferred_temp_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

if rain_check == "Yes":
    preferred_rain_df = preferred_temp_df.loc[(preferred_temp_df["Rain Inches (last 3 hrs)"]>0)]
else:
    preferred_rain_df = preferred_temp_df.loc[(preferred_temp_df["Rain Inches (last 3 hrs)"]==0.0)]

if snow_check == "Yes":
    preferred_cities_df = preferred_rain_df.loc[(city_data_df["Snow Inches (last 3 hrs)"]>0)]
else:
    preferred_cities_df = preferred_rain_df.loc[(city_data_df["Snow Inches (last 3 hrs)"]==0.0)]

    
preferred_cities_df.head()
                                          

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,description,Rain Inches (last 3 hrs),Snow Inches (last 3 hrs)
4,4,Kapaa,US,2020-04-19 15:53:45,22.08,-159.32,69.80,83,90,5.82,overcast clouds,0.0,0.0
13,13,Vila Franca Do Campo,PT,2020-04-19 15:54:55,37.72,-25.43,62.60,59,20,32.21,few clouds,0.0,0.0
34,34,Nichinan,JP,2020-04-19 15:55:54,31.60,131.37,68.00,88,90,19.46,shower rain,0.0,0.0
42,42,Moura,PT,2020-04-19 15:55:56,38.14,-7.45,64.40,52,75,16.11,broken clouds,0.0,0.0
43,43,Hilo,US,2020-04-19 15:54:02,19.73,-155.09,66.99,68,90,9.17,overcast clouds,0.0,0.0


In [6]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "description", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Max Temp,description,Lat,Lng,Hotel Name
4,Kapaa,US,69.80,overcast clouds,22.08,-159.32,
13,Vila Franca Do Campo,PT,62.60,few clouds,37.72,-25.43,
34,Nichinan,JP,68.00,shower rain,31.60,131.37,
42,Moura,PT,64.40,broken clouds,38.14,-7.45,
43,Hilo,US,66.99,overcast clouds,19.73,-155.09,


In [7]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params = {"location": f"{lat},{lng}",
              "radius": 5000,
              "types": "lodging",
              "key": g_key,}

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
print("Complete")    
    


Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Complete


In [10]:
#Create Infobox
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{description} and {Max Temp}°F</dd>
</dl>
"""

hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [11]:
#Create Map
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
#fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [12]:
#Save Results as WeatherPy_vacation.csv
output_data_file = "weather_data/WeatherPy_vacation.csv"
# Export the City_Data into a CSV.
hotel_df.to_csv(output_data_file, index_label="City_ID")